# Write a file

In [ ]:
import numpy as np
import matplotlib.pylab as plt

import h5hep

In [ ]:
# Create a data container
data = h5hep.initialize()

# Create some groups and datasets
# Datasets are like data members of an the group/object

h5hep.create_group(data,'jets',counter='njet')
h5hep.create_dataset(data,['e','px','py','pz'],group='jets',dtype=float)

h5hep.create_group(data,'muons',counter='nmuon')
h5hep.create_dataset(data,['e','px','py','pz'],group='muons',dtype=float)

# Create a single event bucket for us to fill
event = h5hep.create_single_event(data)


## Write some fake data into the file

In [ ]:
nevents = 1000

for i in range(0,nevents):

    # Must do this each time
    h5hep.clear_event(event)

    njets = np.random.randint(0,6)
    event['jets/njet'] = njets

    for n in range(njets):
        event['jets/e'].append(100*np.random.random())
        event['jets/px'].append(100*np.random.random())
        event['jets/py'].append(100*np.random.random())
        event['jets/pz'].append(100*np.random.random())

    nmuons = np.random.randint(0,6)
    event['muons/nmuon'] = nmuons
        
    for n in range(nmuons):
        event['muons/e'].append(100*np.random.random())
        event['muons/px'].append(100*np.random.random())
        event['muons/py'].append(100*np.random.random())
        event['muons/pz'].append(100*np.random.random())

    h5hep.pack(data,event)

print("Writing the file...")

hdfile = h5hep.write_to_file('output.hdf5',data,comp_type='gzip',comp_opts=9)


# Read a file

## Open the file and pull out some data entries

In [ ]:
filename = 'output.hdf5'
data,event = h5hep.load(filename,verbose=False)

print("# of entries: {0}".format(data['nentries']))

x = data['jets/e']

plt.figure()
plt.hist(x,bins=50);

## Read the file so as to only get some datasets (speeds up reading)

In [ ]:
filename = 'output.hdf5'
data,event = h5hep.load(filename,verbose=False,desired_datasets=['jets'])

print("# of entries: {0}".format(data['nentries']))

x = data['jets/e']

plt.figure()
plt.hist(x,bins=50);

## Read only a subset of the entries

In [ ]:
filename = 'output.hdf5'
data,event = h5hep.load(filename,verbose=False,desired_datasets=['jets'],subset=(100,500))

print("# of entries: {0}".format(data['nentries']))

e = data['jets/e']
px = data['jets/px']
py = data['jets/py']
pz = data['jets/pz']

m2 = e**2 - (px**2 + py**2 + pz**2)

plt.figure()
plt.hist(m2,bins=50);

## Read in the data and loop over the entries, a la ROOT

In [ ]:
filename = 'output.hdf5'
data,event = h5hep.load(filename)

print("# of entries: {0}".format(data['nentries']))

nentries = data['nentries']

x = []

for n in range(0,nentries):

    h5hep.unpack(event,data,n=n)

    # Now make use of the "event" bucket
    for i in range(event['muons/nmuon']):
        
        energy = event['muons/e'][i]
        
        if energy>25:
            x.append(event['muons/px'][i])

print(len(x))

plt.figure()
plt.hist(x,bins=100);
